In [1]:
# %cd ./filelists/CUB/
# !ln -s  ../../../CUB_200_2011/images ./images
# %cd ../../

# !ln -s /kaggle/input/caltech-birds-2011-dataset/CUB_200_2011/images ./filelists/CUB/images

# !ln -s /kaggle/input/miniimagenet/miniImageNet/images ./filelists/miniImagenet/images
# !ln -s /kaggle/input/d/arjun2000ashok/tieredimagenet/tiered_imagenet/ ./filelists/tieredImagenet/images
# !ln -s /kaggle/input/stanford-dogs-dataset/images/Images ./filelists/dogs/Images
# !ln -s /kaggle/input/stanford-cars-dataset/cars_train/cars_train ./filelists/cars/images 
# !ln -s /kaggle/input/stanford-car-dataset-by-classes-folder/car_data/car_data ./filelists/cars/images 
!ln -s /kaggle/input/fgvc-aircraft/fgvc-aircraft-2013b/fgvc-aircraft-2013b/data/images ./filelists/aircrafts/images
# !ln -s /kaggle/input/vggflowers/images ./filelists/flowers/images

In [13]:
# Plan : 

# Put a run
# List edits to mkae and tests to perform
# Put BG
# Make edits

'/kaggle/working/fsl_ssl'

In [2]:
!python train.py --dataset cars --train_aug --method protonet --committed --stop_epoch 400

Checkpoint path: ckpts/cars/_resnet18_protonet_aug_5way_5shot_16query_tracking_lr0.0010
Fresh wandb run
wandb: Currently logged in as: multi-input (use `wandb login --relogin` to force relogin)
wandb: wandb version 0.10.31 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade
wandb: Tracking run with wandb version 0.10.2
wandb: Run data is saved locally in wandb/run-20210608_180130-3ppbzp92
wandb: Syncing run major-resonance-66
wandb: ⭐️ View project at https://wandb.ai/multi-input/fsl_ssl
wandb: 🚀 View run at https://wandb.ai/multi-input/fsl_ssl/runs/3ppbzp92
wandb: Run `wandb off` to turn off syncing.

About to start training. Last model and best model will be saved in wandb at every model save. 
 Run save_features.py and test.py after the training completes, with the same arguments
Traceback (most recent call last):
  File "train.py", line 264, in <module>
    train(base_loader, val_loader,  model, start_epoch, stop_epoch, params)
  File "train.py", line 44, i

In [3]:
!python wandb_restore.py --id vba5sgld --path ckpts/aircrafts/_resnet18_protonet_aug_5way_5shot_16query_tracking_lr0.0010/best_model.tar

wandb: Currently logged in as: multi-input (use `wandb login --relogin` to force relogin)
wandb: wandb version 0.10.31 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade
wandb: Tracking run with wandb version 0.10.2
wandb: Run data is saved locally in wandb/run-20210609_045359-vba5sgld
wandb: Resuming run aircrafts protonet
wandb: ⭐️ View project at https://wandb.ai/multi-input/fsl_ssl
wandb: 🚀 View run at https://wandb.ai/multi-input/fsl_ssl/runs/vba5sgld
wandb: Run `wandb off` to turn off syncing.

Checkpoint restored at  ckpts/aircrafts/_resnet18_protonet_aug_5way_5shot_16query_tracking_lr0.0010/best_model.tar
The model's epoch is  399
Please rename it to continue training

wandb: Waiting for W&B process to finish, PID 440
wandb: Program ended successfully.
wandb:                                                                                
wandb: Find user logs for this run at: wandb/run-20210609_045359-vba5sgld/logs/debug.log
wandb: Find internal logs f

In [4]:
# Do again with 200 dimensions
# Do again with 399.tar

!python save_features.py --dataset aircrafts --train_aug --method protonet --committed

Commit after saving features and testing.
checkpoint_dir: ckpts/aircrafts/_resnet18_protonet_aug_5way_5shot_16query_tracking_lr0.0010
USE BN: True
outfile is features/aircrafts/_resnet18_protonet_aug_5way_5shot_16query_tracking_lr0.0010/novel.hdf5
0/40
10/40
20/40
30/40


In [5]:
!python test.py --dataset aircrafts --train_aug --method protonet --committed

Testing involves fine-tuning. Commit after testing.
novel_file features/aircrafts/_resnet18_protonet_aug_5way_5shot_16query_tracking_lr0.0010/novel.hdf5
600 Test Acc = 91.90% +- 0.35%


In [ ]:
!python train.py --dataset aircrafts --train_aug --method baseline --committed

tracking in baseline train: True
USE pre-trained model: False
Checkpoint path: ckpts/aircrafts/_resnet18_baseline_aug_tracking_lr0.0010
Fresh wandb run
wandb: Currently logged in as: multi-input (use `wandb login --relogin` to force relogin)
wandb: wandb version 0.10.31 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade
wandb: Tracking run with wandb version 0.10.2
wandb: Run data is saved locally in wandb/run-20210609_052540-soxdg8to
wandb: Syncing run eager-thunder-70
wandb: ⭐️ View project at https://wandb.ai/multi-input/fsl_ssl
wandb: 🚀 View run at https://wandb.ai/multi-input/fsl_ssl/runs/soxdg8to
wandb: Run `wandb off` to turn off syncing.

About to start training. Last model and best model will be saved in wandb at every model save. 
 Run save_features.py and test.py after the training completes, with the same arguments
Epoch 1 | Batch 50/313 | Loss 4.327444 | Acc Cls 0.030000
Epoch 1 | Batch 100/313 | Loss 4.207465 | Acc Cls 0.024375
Epoch 1 | Batch 15

In [7]:
import torch
torch.cuda.empty_cache()